<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/04-CNN-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Convolutional Neural Networks

En esta notebook implementamos un ejemplo sencillo de una red CNN para el dataset de Fashion MNIST.

<img align="center" width="50%" src="https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/img/Fashion-MNIST.jpg?raw=1"/>

* [Más información sobre el dataset](https://keras.io/api/datasets/fashion_mnist/)
* [Benchmarks](https://paperswithcode.com/sota/image-classification-on-fashion-mnist) en este dataset.

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(f"X_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

Hacemos reescalimiento para tener los valores de intensidad de los pixeles como valores $0\leq x_i \leq 1$:

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

Al ser clasificación multiclase tenemos que codificar los valores de clases a vectores de etiqueta:

In [ ]:
from keras.utils import to_categorical

print(f"y_train shape (valores de clase):\n{y_train.shape}")
num_classes = 10
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
print(f"y_train shape (vectores de clase):\n{y_train.shape}")

Hacemos el reshape para incluir la información sobre el número de canales:

In [ ]:
print(f"x_train shape (antes):\n{x_train.shape}")
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
print(f"x_train shape (después):\n{x_train.shape}")

In [ ]:
#@title Función para graficar las curvas de entrenamiento

import matplotlib.pyplot as plt

def graficar_curvas(history):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].plot(history.history['loss'],label='Training')
    axs[0].plot(history.history['val_loss'],label='Validation')
    axs[0].set_title('Loss')
    axs[0].legend()
    axs[1].plot(history.history['accuracy'],label='Training')
    axs[1].plot(history.history['val_accuracy'],label='Validation')
    axs[1].set_title('Accuracy')
    axs[1].legend()
    fig.show()

Definimos un primer modelo de CNN. Usaremos las capas `Conv2D` para las operaciones de convolución y `MaxPooling2D` para el pooling.

* https://keras.io/api/layers/convolution_layers/convolution2d/
* https://keras.io/api/layers/pooling_layers/max_pooling2d/

Podemos definir el modelo Sequential como usualmente lo hacemos:

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# model = Sequential()
# model.add(Conv2D(filters=16,kernel_size=3,activation='relu',
#                 padding="same", strides=1,
#                  input_shape=(x_train.shape[1], x_train.shape[2], 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# # model.add(Dropout(0.1))
# model.add(Dense(10, activation='softmax'))

# model.summary()

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

O podemos definirlo por medio de una función, que nos permite generar modelos diferentes de forma rápida. Además, de esta manera nos aseguramos de tener modelos *en blanco*, listos para ser entrenados.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

def build_model(num_filtros=16,num_neuronas_densas=[50], dropout=False,
                optimizador='adam'):
    model = Sequential()
    model.add(Conv2D(filters=num_filtros,kernel_size=3,activation='relu',
                        padding="same", strides=1,
                        input_shape=(x_train.shape[1], x_train.shape[2], 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    if num_neuronas_densas is not None:
        for k in num_neuronas_densas:
            model.add(Dense(k, activation='relu'))
            if dropout:
                model.add(Dropout(0.1))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=optimizador, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history = model.fit(x_train, y_train, epochs=30, validation_split=0.2,callbacks=[early_stopping])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
graficar_curvas(history)

Recordemos el rendimiento de los algoritmos de Machine Learning clásico:

<img align="center" width="40%" src="https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/img/ML-Fashion-Mnist.png?raw=1"/>

Podemos entrenar nuevos modelos de forma muy rápida usando la función para generar los modelos:

In [ ]:
from keras.optimizers import Adadelta

#  Efecto de un optimizador en el entrenamiento y rendimiento

# Optimizador lento/malo para este ejemplo:
opt = Adadelta(learning_rate=0.0005)

In [ ]:
model = build_model(num_filtros=4,num_neuronas_densas=None,optimizador='SGD')
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
history = model.fit(x_train, y_train, epochs=30, validation_split=0.2,callbacks=[early_stopping])
print("Evaluación en el conjunto de prueba:")
model.evaluate(x_test, y_test)[1]

In [ ]:
graficar_curvas(history)

Probemos estas combinaciones:

* 16 | 50 | 10
* 8 | 50 con y sin dropout | 10
* 32 | 100 10 | 10
* 64 | 100 100 | 10
* 16 | - | 10

⭕ Práctica

Usando el mismo dataset, implementa las siguientes redes CNN usando como punto de partida la red que hemos implementado, ya sea la versión como función o la *normal* (si te sientes insegura/o respecto a la implementación, no uses el enfoque de función):

* Una red CNN con dos capas convolucionales, en lugar de uno. La segunda capa tendrá las siguientes especificaciones:
 * Una capa convolucional 2D de 8 filtros, el resto de hiperparámetros serán los mismos.
 * Una capa de MaxPooling.
* La red CNN anterior, con las mismas dos capas convolucionales. Cambia la función de activación por `tanh`. ¿Cómo cambian los resultados?
* La red CNN anterior, con las mismas dos capas convolucionales. Cambia el hiperparámetro `padding='valid'`. ¿Qué observas?
* La red CNN anterior, con las mismas dos capas convolucionales. En la parte MLP de la red, agrega 3 capas densas. Usa el número de neuronas en estas capas de tu elección, así como la función de activación en ellas. Experimenta un poco.
* Una red CNN con una capa convolucional con 32 filtros, la parte MLP tendrá un capa oculta de 100 neuronas con activación `relu`. Para el optimizador una un [`SGD`](https://keras.io/api/optimizers/sgd/) con tasa de aprendizaje $0.01$.


* Implementa una red MLP para este mismo problema. Prueba al menos 3 arquitecturas diferentes, ¿cuál fue la mejor opción? ¿cómo se compara con una CNN?
 * No olvides el conjunto de validación.
 * Usa el callback de `EarlyStopping`.
 * Cuida no re-entrenar modelos.

Para este último punto, usa el siguiente código para preparar el dataset:

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print("Shapes al cargar el dataset:")
print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

print("\nShapes al preprocesar el dataset:")
print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

Dudas: mauricio.toledo@unison.mx

Respecto a las dimensiones de salida de una capa convolucional: [wikipedia](https://en.wikipedia.org/wiki/Convolutional_neural_network#Convolutional_layer), [stackoverflow](https://stackoverflow.com/questions/53580088/calculate-the-output-size-in-convolution-layer).